In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import gensim
from gensim import corpora, models
import pyLDAvis.gensim
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jsayd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jsayd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
def get_lda_topics(df, num_topics, num_words):
    # tokenize words
    tokenizer = RegexpTokenizer(r'\w+')
    df["tokens"] = df["title"].apply(tokenizer.tokenize)

    # convert to lowercase
    df["tokens"] = df["tokens"].apply(lambda x: [word.lower() for word in x])
    
    # remove stop words
    stop_words = stopwords.words('english')
    df["tokens"] = df["tokens"].apply(lambda x: [word for word in x if word not in stop_words])
    
    # lemmatize words
    lemmatizer = WordNetLemmatizer()
    df["tokens"] = df["tokens"].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    
    # remove words that appear only once
    all_tokens = sum(df["tokens"], [])
    tokens_once = set(word for word in set(all_tokens) if all_tokens.count(word) == 1)
    df["tokens"] = df["tokens"].apply(lambda x: [word for word in x if word not in tokens_once])

    # remove the token 'iphone'
    df["tokens"] = df["tokens"].apply(lambda x: [word for word in x if word != "iphone"])

    # remove one character tokens
    df["tokens"] = df["tokens"].apply(lambda x: [word for word in x if len(word) > 2])
    
    # create dictionary and corpus
    dictionary = corpora.Dictionary(df["tokens"])
    corpus = [dictionary.doc2bow(text) for text in df["tokens"]]
    
    # create LDA model
    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=42)
    
    # print topics
    topics = lda.show_topics(num_topics=3, num_words=10, formatted=False)
    for topic_id, words in topics:
        print(f"Topic {topic_id + 1}: {', '.join(word[0] for word in words)}")

    # visualize topics
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
    return vis

In [7]:
pre_release = pd.read_csv('.\\pre_release.csv')
release = pd.read_csv('.\\release.csv')

In [9]:
pre_vis = get_lda_topics(pre_release, 3, 10)

Topic 1: apple, new, face, event, cnet, hand, news, best, everything, comparison
Topic 2: plus, apple, galaxy, samsung, note, hand, leak, talk, tech, camera
Topic 3: buy, galaxy, reason, plus, note, get, one, know, thing, phone


c:\Users\jsayd\anaconda3\envs\lds\Lib\site-packages\joblib\_utils.py:37: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
c:\Users\jsayd\anaconda3\envs\lds\Lib\site-packages\joblib\_utils.py:37: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
c:\Users\jsayd\anaconda3\envs\lds\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  return node.n
c:\Users\jsayd\anaconda3\envs\lds\Lib\site-packages\joblib\_utils.py:37: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
c:\Users\jsayd\anaconda3\envs\lds\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.009790 -0.101773       1        1  41.487444
0     -0.085986  0.059109       2        1  33.857816
2      0.095776  0.042664       3        1  24.654741, topic_info=          Term       Freq      Total Category  logprob  loglift
0          buy  13.000000  13.000000  Default  30.0000  30.0000
8        apple  31.000000  31.000000  Default  29.0000  29.0000
2       reason   5.000000   5.000000  Default  28.0000  28.0000
14        note  11.000000  11.000000  Default  27.0000  27.0000
22         get   5.000000   5.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
30         top   1.253710   5.813415   Topic3  -4.5516  -0.1339
45         new   1.297493   8.216230   Topic3  -4.5173  -0.4455
55  comparison   1.284255   9.100889   Topic3  -4.5276  -0.5580
32     feature   1.238940   7.472885   Topic3  -4.5635  -0.3968
5         face   1.208938   6.568901   Topic3  -4.5880  -0.2924

[149 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
39        1  0.171768      2017
39        2  0.515305      2017
39        3  0.343537      2017
89        1  0.804076    airtel
89        2  0.402038    airtel
...     ...       ...       ...
71        1  0.403651     watch
71        2  0.403651     watch
88        1  0.801443  whatsapp
81        2  0.917039  wireless
91        2  0.811415      work

[179 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])

In [10]:
get_lda_topics(release, 3, 10)

Topic 1: review, case, test, apple, camera, plus, speed, galaxy, best, samsung
Topic 2: unboxing, apple, first, camera, daily, pocketnow, comparison, feature, setup, review
Topic 3: test, speed, apple, note, galaxy, comparison, pixel, camera, tip, drop


c:\Users\jsayd\anaconda3\envs\lds\Lib\site-packages\joblib\_utils.py:37: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
c:\Users\jsayd\anaconda3\envs\lds\Lib\site-packages\joblib\_utils.py:37: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
c:\Users\jsayd\anaconda3\envs\lds\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  return node.n
c:\Users\jsayd\anaconda3\envs\lds\Lib\site-packages\joblib\_utils.py:37: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
c:\Users\jsayd\anaconda3\envs\lds\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.070611  0.058376       1        1  40.661020
0      0.020848 -0.084242       2        1  31.446365
1     -0.091459  0.025866       3        1  27.892616, topic_info=        Term       Freq      Total Category  logprob  loglift
12    review  50.000000  50.000000  Default  30.0000  30.0000
68      case  37.000000  37.000000  Default  29.0000  29.0000
18      test  71.000000  71.000000  Default  28.0000  28.0000
6      first  15.000000  15.000000  Default  27.0000  27.0000
1   unboxing  40.000000  40.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
12    review   6.637334  50.147513   Topic3  -4.2402  -0.7455
31       new   4.385178  15.401261   Topic3  -4.6547   0.0206
59      face   4.272564  17.484877   Topic3  -4.6807  -0.1323
87       top   4.342554  19.146314   Topic3  -4.6645  -0.2068
62    google   3.877531  13.368120   Topic3  -4.7777   0.0391

[196 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
200       2  0.624067       2017
88        2  0.342317       2018
88        3  0.570529       2018
43        1  0.704998  accessory
43        2  0.100714  accessory
...     ...       ...        ...
90        2  0.628941       week
120       1  0.273153   wireless
120       2  0.136576   wireless
120       3  0.546306   wireless
149       1  0.737342     xperia

[248 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])